In [1]:
import requests
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd 

In [2]:
def embajadas():
    response = requests.get('https://datos.madrid.es/egob/catalogo/201000-0-embajadas-consulados.json')
    results = response.json()
    embajadas = pd.json_normalize(results['@graph'])
    embajadas["Dirección"] = embajadas["address.street-address"]
    return embajadas.to_json("../data/embajadas.json", orient='records')
def estaciones():
    estaciones = pd.read_json("../data/bicimad2.json", orient='records')
    geometry_coordinates = estaciones["geometry_coordinates"].str.split(expand=True)
    geometry_coordinates.columns = ['LONGITUD', 'LATITUD']
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace("[","")
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace(",","")
    geometry_coordinates['LATITUD'] = geometry_coordinates['LATITUD'].str.replace("]","")
    estaciones = pd.concat([stations, geometry_coordinates], axis=1)
    return estaciones.to_json("../data/estaciones.json", orient='records')


In [5]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

def distance_meters(mercator_start, mercator_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    return mercator_start.distance(mercator_finish)
    


In [6]:
embajadas = pd.read_json('../data/embajadas.json')
estaciones = pd.read_json('../data/estaciones.json')

In [9]:
response = requests.get('https://datos.madrid.es/egob/catalogo/201000-0-embajadas-consulados.json')
results = response.json()
pd.json_normalize(results(['@graph']))

TypeError: 'dict' object is not callable

In [10]:
embajadas["mercator_start"] = embajadas.apply(lambda x: to_mercator(x["LATITUD"], x["LONGITUD"]), axis=1)
estaciones["mercator_finish"] = estaciones.apply(lambda x: to_mercator(x["LATITUD"], x["LONGITUD"]), axis=1)


In [11]:
estaciones_clean = estaciones[["name","address","LONGITUD","LATITUD", "mercator_finish"]]
estaciones_clean = estaciones_clean.rename(columns={"name": "BiciMAD station", "address": "Station location", "LONGITUD": "Longitud_final", "LATITUD": "Latitud_final"})


In [12]:
embajadas_clean = embajadas[["NOMBRE","Dirección","LONGITUD","LATITUD", "mercator_start"]]
embajadas_clean = embajadas_clean.rename(columns={"NOMBRE": "Place of interest", "Dirección": "Place address", "LONGITUD": "Longitud_inicial", "LATITUD": "Latitud_inicial"}, errors="raise")


In [13]:
df_resultado = pd.merge(embajadas_clean,estaciones_clean, how='cross')

In [18]:
df_resultado["Distancia"] = df_resultado.apply(lambda x: distance_meters(x["mercator_start"], x["mercator_finish"]), axis=1)
df_resultado


,Place of interest,Place address,Longitud_inicial,Latitud_inicial,mercator_start,BiciMAD station,Station location,Longitud_final,Latitud_final,mercator_finish,Distancia
0,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Puerta del Sol A,Puerta del Sol nº 1,-3.701834,40.417214,POINT (4499223.648366921 -412373.2851572837),1595.648926
1,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Puerta del Sol B,Puerta del Sol nº 1,-3.701603,40.417313,POINT (4499234.670122574 -412347.498528782),1570.712528
2,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Miguel Moya,Calle Miguel Moya nº 1,-3.705842,40.420589,POINT (4499599.340516399 -412820.3206106317),1697.402416
3,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3,-3.706917,40.430294,POINT (4500679.707306497 -412940.3068148262),1606.478855
4,Consulado de Argentina,CALLE FERNANDO EL SANTO Nº15,-3.692666,40.428216,POINT (4500448.446385445 -411350.5607033509),Malasaña,Calle Manuela Malasaña nº 5,-3.702587,40.428552,POINT (4500485.866677178 -412457.3286497012),1107.400364
...,...,...,...,...,...,...,...,...,...,...,...
42763,Embajada del Nepal,CALLE GENERAL ORAA Nº3,-3.687905,40.436031,POINT (4501318.352334742 -410819.4374695785),Ciudad Universitaria 1,Avenida de la Complutense (Metro Ciudad Univer...,-3.726990,40.443750,POINT (4502177.655770458 -415179.5220676512),4443.955456
42764,Embajada del Nepal,CALLE GENERAL ORAA Nº3,-3.687905,40.436031,POINT (4501318.352334742 -410819.4374695785),Ciudad Universitaria 2,Avenida de la Complutense (Metro Ciudad Univer...,-3.726930,40.443420,POINT (4502140.920338497 -415172.8287427756),4430.421391
42765,Embajada del Nepal,CALLE GENERAL ORAA Nº3,-3.687905,40.436031,POINT (4501318.352334742 -410819.4374695785),Facultad Biología,Calle José Antonio Novais frente al nº 12,-3.727295,40.448332,POINT (4502687.743941171 -415213.4906984274),4602.492493
42766,Embajada del Nepal,CALLE GENERAL ORAA Nº3,-3.687905,40.436031,POINT (4501318.352334742 -410819.4374695785),Facultad Derecho,Avenida Complutense nº 23,-3.729370,40.451090,POINT (4502994.74083288 -415445.024322674),4919.993103


In [ ]:
df_minimo = df_resultado[df_resultado["Place of interest"] == input('Pon un lugar de interes: ')]